In [18]:
import torch
from transformers import BertTokenizer, BertModel

In [19]:
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
import logging

In [21]:
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [22]:
import pandas as pd
import numpy as np

df_items = pd.read_csv('items-Copy1.csv')

df_items = df_items.replace(np.nan, '', regex=True)

In [23]:
np.random.seed(500)

In [24]:

descriptions_list = [] # holds list of descriptions
labels_list_numerical = []

# binary classification so either anger or not
for i, row in df_items.iterrows():
    row['description'] = '[CLS] ' + row['description'] + '[SEP]'
    descriptions_list.append(row['description']) # add description
    
    row['tag'] = row['tag'].lower()
    
    if 'anger' in row['tag']:
        labels_list_numerical.append(1)
    else:
        labels_list_numerical.append(0)

df_marked_text = pd.DataFrame() # create empty dataframe
df_marked_text['description'] = descriptions_list # append descriptions_list to df with column header 'description'
df_marked_text['label numerical'] = labels_list_numerical
df_marked_text

,description,label numerical
0,[CLS] Black American racial experience is real...,0
1,[CLS] The best costume for Halloween worth pos...,1
2,[CLS] Keep on whining and crying for your pres...,1
3,"[CLS] End the whining and crying, end the riot...",1
4,[CLS] Black girls are the definition of nation...,0
...,...,...
3007,"[CLS] New ideas, old values. Like us![SEP]",0
3008,[CLS] Secured borders are a national priority....,0
3009,[CLS] Secured borders should be a top priority...,0
3010,[CLS] Bernie for president![SEP],0


In [25]:
list_of_array_embeddings = []
token_vecs_cat = []

counter = 1

model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
model.eval()

for i, row in df_marked_text.iterrows():
    
    print('Embedding entry: ' + str(counter) + '...')
    print('i: ' + str(i))
    
    # this entry is too long for the torch model, I decided to split it in half
    if counter == 2139:
        row['description'] = row['description'][:len(row['description'])//2] + '[SEP]'
        print(row['description'])
        tokenized_text = tokenizer.tokenize(row['description'])
    else:
        print(row['description'])
        tokenized_text = tokenizer.tokenize(row['description'])
        
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segment_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segment_ids])
    
    with torch.no_grad():
        
        outputs = model(tokens_tensor, segments_tensors)
        
        hidden_states = outputs[2]
        
    token_vecs = hidden_states[-2][0]
    
    sentence_embedding = torch.mean(token_vecs, dim=0)
    
    list_of_array_embeddings.append(sentence_embedding.numpy())
    
    counter += 1

Embedding entry: 1...
i: 0
[CLS] Black American racial experience is real. We support the 2nd ammendment
for our safety.[SEP]
Embedding entry: 2...
i: 1
[CLS] The best costume for Halloween worth posting. Power to the People![SEP]
Embedding entry: 3...
i: 2
[CLS] Keep on whining and crying for your president, ooh, who wasn't elected.
Imma take care of my kids.

I'll keep grinding for them, to make sure the get the best education.
To ensure they dress well and eat healthy food.
And I'm gon tell you to do the same thing.

Don't waste time on an "afterparty" you wouldn't have been invited to!

#word #wakeup #organize #joshuabeal #altonsterling #TerenceCrutCher
#westandtogether #blacklivesmatter #unite4justice #hovem ber8
#electi0h52016 #black #melanin #blackisbeautiful #blacklivesmatter
#blackgirlmagic #USA #blackahdproud #democrat #republioan
#Africanamerican #unite4justice #vote #breakingnews[SEP]
Embedding entry: 4...
i: 3
[CLS] End the whining and crying, end the riots with these same

Embedding entry: 25...
i: 24
[CLS] Are you unhappy with our new president his bigoted agenda? Looking for
sweatshirt or hoodie to prove that?

Click the link down below and find a hoodie that‘s just for you.

https://represent.com/not-my-president-1 Stop Trump and his bigoted agenda!
Not My President BM[SEP]
Embedding entry: 26...
i: 25
[CLS] The cop beat this man like he was a runaway slave.[SEP]
Embedding entry: 27...
i: 26
[CLS] America is far from stamping on racism problem. Unfortunately, the skin color
can determine a whole range of variables in life.[SEP]
Embedding entry: 28...
i: 27
[CLS] Institutional racism takes a hold of our schools as white students of the
University of North Dakota make a mockery of Black people. Institutional
Racism: White Students Slam BLM | BM BM[SEP]
Embedding entry: 29...
i: 28
[CLS] Have seen good ones, but this is so far, in my opinion, the best
#mannequinchallenge. Hands down![SEP]
Embedding entry: 30...
i: 29
[CLS] Montgomery County middle school

Embedding entry: 53...
i: 52
[CLS] There you go, girl![SEP]
Embedding entry: 54...
i: 53
[CLS] I stopped watching tv for new, cus i can't take all this bs. i see my people do!
| wana put this out straight, once and for all!
We all say it's a whiteman's world in the US. with white-favouring-rules and
everything, right?
So what makes you want to be a part of this system, in other terms, be a
slave to this system?

Why in the heck would you all be routing and rioting and fighting over and for
a two-party government system that cares less about you?

Damn, we all agreed with Kaepernick when he took the knee for us all, and
now we all, outta fear join white democrats to riot Trump?

Like wtf happened to we rising above the government system to improve our
lives and not joining this whole racist bias system?

Damn, that's sad!

#wakeup #organize #joshuabeal #altonsterling #TerenceCrutCher
#westandtogether #blacklivesmatter #unite4justice #hovem ber8
#elections2016 #black #melanin #blackisbea

Embedding entry: 77...
i: 76
[CLS] Staying Woke,bui|ding-up our Black communities,up|ifting Black people. Join
us![SEP]
Embedding entry: 78...
i: 77
[CLS] Donald John Trump was elected the 45th president of the United States on

Tuesday! Today some massive crowds of libtards marched in NYC against
election of Donald Trump.

Come to Trump Tower (725 5th Ave) on Saturday to show Mr. Trump there
actually ARE people in NY who voted for him! Support Trump in NYC[SEP]
Embedding entry: 79...
i: 78
[CLS] CHARLESTON COP WHO FATALLY SHOT BLACK MAN ASKS JUDGE TO
DROP CHARGES, CITING DOUBLE RISK

North Charleston police officer Michael Slager, who shot and killed 50-year-
old Scott after an April 2015 traffic stop, and his attorneys urged the court to
drop murder Charge due to the double jeopardy that Slager faced before the
shooting. According to ex-cop‘s attorneys, their defendant acted in self-
defense despite the fact that Scott was unarmed and Slager fired eight times,
hitting Scott five time

Embedding entry: 106...
i: 105
[CLS] Once we recognize what it is we are feeling, once we recognize we can feel
deeply, love deeply, can feel joy, then we will demand that all parts of our
lives produce that kind of joy
#black #woman #quotes #powerful #woke #panafrican[SEP]
Embedding entry: 107...
i: 106
[CLS] If your black matters, Click
https://represent.com/my-black-matters?store=black-matters-us[SEP]
Embedding entry: 108...
i: 107
[CLS] Joshua was born in Chicago, Illinois, and raised in Indianapolis. Two months
before his death, he proposed to his longtime girlfriend and the mother of two
sons.

Joshua Beal had come to Chicago to serve as a pallbearer in the funeral of
his cousin Marcus Washington who was fatally shot on October 27, 2016 in
Indianapolis at the age of 26.

Beal was fatally shot after he and several relatives were involved in what
authorities called a road rage incident with an off-duty police officer and an off-
duty firefighter.

The relatives along with others in

...

Embedding entry: 2869...
i: 2868
[CLS] Killary Clinton will never understand what it feels like to lose the person you
love for the sake of your country. Honoring the high cost paid by so many
families to protect our freedom. Buy a T-shirt - help a veteran: Veterans USA
Funnylnst[SEP]
Embedding entry: 2870...
i: 2869
[CLS] The law is the law. If I commit a crime, I will get ticketed or arrested. Why do
illegal aliens break the law, but our government harbors them? Illegals have
more benefits than those who shed their blood for America and our freedom.
It makes me sick...[SEP]
Embedding entry: 2871...
i: 2870
[CLS] Border Patrol agents in South Texas arrested an illegal alien from Honduras
that had previously been deported and convicted of Rape Second Degree.

Thanks to Obama's and Hillary's policy, illegals come here because they wait
for amnesty promised. The wrong course had been Chosen by the American
government, but all those politicians are too far from the border to see who
actua

Embedding entry: 2890...
i: 2889
[CLS] More than 2,000 illegal aliens commit sex assaults every year in Texas alone.
Americans would really feel safer, if the government Closes the borders and
deports all illegal aliens.[SEP]
Embedding entry: 2891...
i: 2890
[CLS] The safety, freedom, and dignity of all people must be safeguarded. Terrorism
and anti-Mus Safe Space for Muslim Neighborhood[SEP]
Embedding entry: 2892...
i: 2891
[CLS] Due to the town of Twin falls, Idaho, becoming a center of refugee
resettlement, which led Citizens before refugees[SEP]
Embedding entry: 2893...
i: 2892
[CLS] Ohh..say no more! Fort Campbell says about 400 soldiers with the 101st
Airborne Division are deploying to Iraq this summer. And the soldiers from the

2nd Brigade Combat Team will deploy as part of an increase in authorized
troop levels in Iraq.

Obama still drafts our guys into the army and they go to the Middle East. But
his key campaign promise was to withdraw troops! It’s Clear that we come

throug

Embedding entry: 2924...
i: 2923
[CLS] The leader of radical religious group Nation of Islam, Louis Farrakhan,
announced that 20th anniversary Million Man March rally will take place on
October 10, 2015 in Washington, DC, at the National Mall. Organizers of the
Million Man March call forjustioe, standing against the police and for blacks,
Latinos and incarcerated.

They say racism rises among the police officers, but in fact it rises among
radical groups. Only black activists say “black lives matter”, forgetting the fact
thatjustice is when all lives matter.

Americans are not against blacks, Latinos, etc. But if somebody breaks the
law, he should be punished despite of his color or religion. If somebody goes
over the speed limit, he should be ticketed. If somebody breaks the
immigration law, he should be deported. If somebody kills a man, he should
be imprisoned.

Given the fact that two Islamic extremists who were arrested one year ago in
Ferguson belong to the Nation of Islam, I can

Embedding entry: 2966...
i: 2965
[CLS] This year had some good ones, but you already know them, so here's the list
of the movies that were not so good DudeTube The Worst Movies of 2016
2016-12-302016-12-30 0 Comment This year had some good ones, but you
already know them, so here's the list of the movies that were not so good
Share on FacebookShare on Twitter The Worst Movies of 2016 — DuDeers
dudeers.com[SEP]
Embedding entry: 2967...
i: 2966
[CLS] Heritage, not hate. The South will rise again! South United[SEP]
Embedding entry: 2968...
i: 2967
[CLS] Shop our full range of sale men's and women‘s t-shirts. We feature new
amazing limited edition t-shirtS with awesome print.

https://www.facebook.com/pg/bIacktivists/shop/?rid=128371547505950&rt=6
www.facebook.com facebook.com[SEP]
Embedding entry: 2969...
i: 2968
[CLS] Welcome to the future.
I'm very proud of all our Black youth out there making a difference by getting
knowledge. The future is bright and our liberation is certain, just st

Embedding entry: 2986...
i: 2985
[CLS] Do you remember the other Nice Peter vs Epiclloyd rap battle that took place
5 years ago?? Here's the new one! DudeTube Epic Rap Battles Of History
Season Finale. 2017-01-11 0 Comment Do you remember the other Nice
Peter vs Epiclloyd rap battle that took place 5 years ago?? Share on
FacebookShare on Twitter More ERB you can find on their youtube Channel.
Epic Rap Battles Of History Season Finale. — DuDeers dudeers.com[SEP]
Embedding entry: 2987...
i: 2986
[CLS] Join us to study your real history and get the power from your roots. Stay
woke and natural! Nefertiti's Community[SEP]
Embedding entry: 2988...
i: 2987
[CLS] People really need to understand[SEP]
Embedding entry: 2989...
i: 2988
[CLS] If you are Native, love and admire Native culture, join us! Native Americans
United[SEP]
Embedding entry: 2990...
i: 2989
[CLS] Protect the 2nd. Without it, you won't have any others! Join! Defend The 2nd[SEP]
Embedding entry: 2991...
i: 2990
[CLS] Black Mill

In [26]:
len(list_of_array_embeddings)

3012

In [27]:
list_of_array_embeddings[2140]

array([ 1.72993436e-01,  3.68748158e-02,  3.88787277e-02, -4.56990451e-02,
        2.45266065e-01,  1.33522183e-01,  1.62904546e-01, -2.76506364e-01,
       -5.31050682e-01,  2.56400347e-01,  2.81690717e-01,  2.78708458e-01,
        1.02372855e-01,  4.35233722e-03, -3.57697666e-01,  3.90351653e-01,
        2.02697098e-01,  9.42187198e-03, -6.41120523e-02,  4.10709321e-01,
       -4.04452324e-01,  2.52118468e-01,  1.42877102e-01, -3.96912582e-02,
        6.96525574e-02, -1.66717529e-01, -2.36844271e-01, -3.28568131e-01,
       -2.84528553e-01,  1.29297867e-01,  3.27400118e-01,  1.64461672e-01,
        2.75086075e-01, -8.28289986e-01,  1.62085763e-03, -3.60635966e-01,
        7.85499036e-01, -1.01328887e-01,  1.20279282e-01,  1.70776919e-01,
        2.49573648e-01, -5.79508066e-01,  6.15046620e-01, -2.14331269e-01,
        3.04101855e-01,  4.69262078e-02, -2.54572392e-01, -2.03402549e-01,
        3.67515475e-01, -4.03291792e-01, -6.08592033e-02,  4.15424049e-01,
       -2.17658028e-01, -

In [30]:
list_to_save = []

dataset_order = 0

for i, arr in enumerate(list_of_array_embeddings):
    list_to_save.append(np.array([[dataset_order], [labels_list_numerical[i]], [arr]], dtype=object))
    
    dataset_order += 1

In [35]:
list_to_save[4]

array([[4],
       [0],
       [array([ 9.66176465e-02, -4.45756763e-01, -1.73208892e-01, -5.44688851e-02,
        1.29542887e-01, -1.00082479e-01,  2.38155380e-01,  8.51806879e-01,
       -7.08985984e-01, -4.17903274e-01,  3.77088875e-01, -2.23572850e-01,
       -1.41403958e-01,  2.48803005e-01, -4.44450885e-01,  3.58936399e-01,
        2.45553646e-02,  1.43679485e-01, -1.20038681e-01,  4.35859598e-02,
       -4.27865893e-01,  1.00315988e-01, -3.12289655e-01,  3.86134714e-01,
        1.71753213e-01, -3.49704713e-01,  5.65134101e-02, -6.99512437e-02,
       -3.26924562e-01,  2.73340911e-01,  1.93924651e-01,  1.25910267e-01,
       -3.78501773e-01, -9.30492058e-02, -1.92443673e-02, -5.71832597e-01,
        3.46313715e-01, -4.55222994e-01,  1.58933029e-01,  2.36084342e-01,
       -5.16008556e-01, -4.69671339e-01,  1.00291520e-03, -1.00844614e-01,
       -4.90418404e-01, -8.17529634e-02, -4.61083323e-01,  3.54941994e-01,
        4.85995054e-01, -4.04676288e-01, -2.92520612e-01, -1.4243449

In [36]:
np.savez("berd_word_embeddings.npz", *list_to_save) 

In [40]:
data = np.load("berd_word_embeddings.npz", allow_pickle=True) 

data.files
data['arr_0'].shape # last element key

(3, 1)

In [42]:
data['arr_1']

array([[1],
       [1],
       [array([ 1.05588585e-01, -8.10484171e-01,  5.13743460e-01,  6.63671717e-02,
       -1.21236928e-01, -1.95709746e-02,  4.16527063e-01,  5.96855104e-01,
       -4.82012242e-01, -2.65671164e-01, -7.46658654e-04, -1.81513354e-01,
        1.47390381e-01,  1.58558547e-01, -3.65100563e-01,  3.53429675e-01,
       -2.57677108e-01,  1.86351478e-01,  2.33519301e-01,  1.54551556e-02,
        2.61451900e-01,  7.13624209e-02, -9.80269983e-02,  3.36883171e-03,
        7.81696141e-02, -2.84864396e-01, -1.66727602e-01,  2.21870795e-01,
       -3.17563027e-01,  1.44902974e-01, -1.28282188e-02,  3.99919569e-01,
       -3.81826051e-02, -1.45083293e-01, -4.86813277e-01, -4.09382552e-01,
        1.26801446e-01, -1.18563734e-01,  2.41490211e-02,  2.19370201e-01,
       -5.30007899e-01, -3.54415596e-01,  6.54862761e-01, -7.58476779e-02,
       -3.12530965e-01, -2.02445939e-01,  1.06490582e-01,  1.10342197e-01,
        3.47436756e-01, -5.08832335e-01,  6.52399957e-02,  6.9295749

In [43]:
np.array(data.files).shape

(3012,)